In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Cambiar el directorio de trabajo
%cd '/content/drive/My Drive/tfg/tfg/'

# Mostrar el directorio actual para confirmar el cambio
directorio_actual = %pwd
print("Directorio actual:", directorio_actual)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/tfg/tfg
Directorio actual: /content/drive/My Drive/tfg/tfg


In [2]:
import os
import pandas as pd
path_actual = os.getcwd()
subdirectorio = 'datas'
file_train = 'df_train.csv'
path_train = os.path.join(path_actual, subdirectorio, file_train)

In [3]:
import pandas as pd

df_train = pd.read_csv(path_train, index_col=None)


In [4]:
df_train = df_train.drop(['trueE'],axis=1)

In [5]:
import numpy as np

def filtrado_datos_concatenados(df, N):
    # Filtrado inicial por PDGcode
    codes = df['PDGcode'].unique()
    kaones = []
    labels = []

    df_filtrado = df[df['PDGcode'].isin(codes)]

    # Identificar todos los eventIDs únicos
    unique_eventIDs = df_filtrado['eventID'].unique()

    # Iterar sobre cada eventID único
    for eventID in unique_eventIDs:
        # Filtrando el DataFrame por eventID
        df_evento = df_filtrado[df_filtrado['eventID'] == eventID]

        # Ordenando por hitTime en orden descendente
        df_evento_ordenado = df_evento.sort_values(by='hitTime', ascending=False)

        # Ajustando el tamaño de los hits a N
        num_hits = min(len(df_evento_ordenado), N)

        # Inicializando listas para cada tipo de dato
        hitX_values, hitY_values, hitZ_values, hitInteg_values = [], [], [], []

        # Recolectando los valores separadamente
        for i in range(num_hits):
            hitX_values.append(df_evento_ordenado.iloc[i]['hitX'])
            hitY_values.append(df_evento_ordenado.iloc[i]['hitY'])
            hitZ_values.append(df_evento_ordenado.iloc[i]['hitZ'])
            hitInteg_values.append(df_evento_ordenado.iloc[i]['hitInteg'])

        # Concatenando los valores recolectados
        hit_values = hitX_values + hitY_values + hitZ_values + hitInteg_values

        # Rellenando con ceros si hay menos hits que N
        if len(hit_values) < 4 * N:  # 4 tipos de datos * N hits
            hit_values.extend([0] * (4 * N - len(hit_values)))

        # Asegurarse de limitar a 4*N para considerar casos donde se excede
        kaones.append(hit_values[:4 * N])
        labels.append(df_evento.iloc[0]['PDGcode'])  # Tomar el PDGcode del primer hit como etiqueta

    return np.array(kaones), np.array(labels)

In [6]:
N=60

In [7]:
matriz_train, etiquetas_train = filtrado_datos_concatenados(df_train, N)

In [8]:
file_valid = 'df_valid.csv'
path_valid = os.path.join(path_actual, subdirectorio, file_valid)

df_valid = pd.read_csv(path_valid, index_col=None)
df_valid = df_valid.drop(['trueE'],axis=1)


In [9]:
matriz_valid, etiquetas_valid = filtrado_datos_concatenados(df_valid, N)

In [10]:
valores_unicos = np.unique(etiquetas_train)

print(valores_unicos)

[211. 321.]


In [11]:
valores_unicos_pdgcode = df_train['PDGcode'].unique()

print(valores_unicos_pdgcode)

[211 321]



Este modulo será el encargado de entrenar los modelos de machine learning y ver los resultados que muetran. Además elegiremos los mejores hiperparametros para los modelos ya que estos modelos serán cargados de la biblioteca Scikit-learn.

Comprobaremos las diferencia de cada modelo para ver que modelo da mejores resultados.

Los modelos que vamos a usar son los siguientes:

*   **Regresión logistica:** Es un modelo lineal que es simple pero eficaz para problemas de clasificación binaria. Este modelo nos dará el punto de partida.

*   **Arboles de decisión:** Es un modelo versatil que puede usarse tanto para clasificación como para regresión. Los arboles de decisión son fáciles de interpretar y pueden capturar relaciones no lineales entre caracterisiticas.

*   **Random Forest:** Un ensamble de arboles de decisión. Es más robusto y preciso que un árbol unico de decisión debido a que reduce el riesgo de sobreajuste mediante el promedio de múltiples árboles

*   **SVM:** Es efectivo en espación de alta dimensión y casos donde el número de dimensiones es mayor que el número de muestras. Es útil tanto para la clasificación lineal como no lineal mediante el uso de diferentes kerneles.

*   **GBM:** Es una técnica de ensamble que construye modelos de forma secuencial, cada nuevo modelo corrige errores del modelo anterior.

*  **k-NN:** Es un modelo simple que clasifica puntos basándose en la mayoría de votos de sus k vecinos más cercanos. Es útil cuando las relaciones entre caracteristicas son complejas y no lineales.

*  **XGBoost:** es una implementación optimizada de árboles de decisión aplicados al algoritmo de Gradient Boosting. Es muy eficaz por su eficiencia en la ejecución.

*   **LightGBM:** es otro algoritmo de Gradient Boosting desarrollado por Microsoft que utiliza árboles basados en histogramas para el modelado.Utiliza la técnica de construcción de árboles basada en histogramas, que agrupa valores continuos en bins discretos



In [12]:
from sklearn.linear_model import LogisticRegression


model = LogisticRegression()

model.fit(matriz_train, etiquetas_train)



predictions = model.predict(matriz_valid)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
# Calcular el número de predicciones correctas
correctas = np.sum(predictions == etiquetas_valid)

# Calcular el porcentaje de precisión
precision = (correctas / len(predictions)) * 100

print(f'La precisión del modelo es: {precision}%')

La precisión del modelo es: 98.70422535211267%
